# Parameter definition
Choose all the settings for the database creation. Input, output dir...

In [1]:
#TODO: Change absolute to relative paths for input and output dir.
import os

home = os.path.expanduser("~")

db_name= 'test_video_dataset_chokepoint1' #'colab_main_dataset'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['enfsi', 'enfsi2015'] # 'lfw', 'scface',  'enfsi', 'enfsi2015' 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn','mtcnn_serfiq'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',
save_in_drive = False # To work in google colab

## SERFIQ model

Create SERFIQ model.

In [2]:
from sql_face.serfiq import get_serfiq_model
serfiq = get_serfiq_model()

/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[20:10:51] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.8.0. Attempting to upgrade...
[20:10:51] ../src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!
/home/andrea/anaconda3/envs/sql-face/lib/python3.7/site-packages/mxnet/gluon/block.py:1784: UserWarning: Cannot decide type for the following arguments. Consider providing them as input:
	data: None
  input_sym_arg_type = in_param.infer_type()[0]
[20:10:52] ../src/base.cc:79: cuDNN lib mismatch: linked-against version 8100 != compiled-against version 8101.  Set MXNET_CUDNN_LIB_CHECKING=0 to quiet this warning.
[20:10:59] ../src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v0.8.0. Attempting to upgra

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [4]:
from sql_face.sqldb import SQLDataBase

In [5]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

Creating Db file at /home/andrea/lr_video_project/sql_database/test_video_dataset_chokepoint1.db


In [ ]:
database.__dict__

In [6]:
database.create_tables(serfiq=serfiq)

Creating Cropped Images for detector mtcnn:   0%|          | 0/4433 [00:00<?, ?it/s]2022-12-08 20:12:59.729615: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:12:59.730315: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:12:59.730588: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-08 20:12:59.731915: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To ena

1/1 [==============================] - 0s 281ms/step


Creating Cropped Images for detector mtcnn:   0%|          | 1/4433 [00:04<5:21:05,  4.35s/it]2022-12-08 20:13:04.172746: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 564.44MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-12-08 20:13:04.173255: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 564.44MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-12-08 20:13:04.202437: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 290.24MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if m

1/1 [==============================] - 0s 167ms/step


2022-12-08 20:13:04.235024: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 507.66MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-12-08 20:13:04.235058: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 507.66MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-12-08 20:13:04.395846: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 290.37MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2022-12-08 20:13:04.395886: W tensorflow/core/common_runtime/bfc_al

...

1/1 [==============================] - 0s 41ms/step


Creating Cropped Images for detector mtcnn:   0%|          | 13/4433 [00:56<10:56:25,  8.91s/it]

2/2 [==============================] - 0s 232ms/step


: 

: 

In [ ]:
database.update_tables(attributes_to_update, serfiq = serfiq)